In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'daseCV/assignments/assignment3/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/daseCV/datasets/
!bash get_datasets.sh
%cd /content

# Style Transfer
该作业中 style transfer 的实现源于以下论文 ["Image Style Transfer Using Convolutional Neural Networks" (Gatys et al., CVPR 2015)](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf).

总体思路是拿到两张图片，其中一张图片提供内容，另一张图片提供“风格”，以此生成一张新的图片。为此我们需要设计一种损失函数，该损失函数能够分别衡量生成图片的内容与“风格”与源图的匹配程度，然后据此对生成图像的像素执行梯度下降。

我们用在ImageNet上预训练的比较小巧的 [SqueezeNet](https://arxiv.org/abs/1602.07360) 来提取图像特征。这里你可以选择任何网络，我们选择 SqueezeNet 的原因在于其小巧与高效的特点。

以下为该作业最后生成的图像示例：

![caption](example_styletransfer.png)


# Part 0: Setup

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import PIL

import numpy as np

import matplotlib.pyplot as plt

from daseCV.image_utils import SQUEEZENET_MEAN, SQUEEZENET_STD
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

我们为你提供一些处理图像的辅助函数，因为在这部分作业中，我们处理的是真实的JPEG图片，而不是CIFAR-10数据集。

In [ ]:
from daseCV.style_transfer_pytorch import preprocess, deprocess, rescale, rel_error, features_from_img

CHECKS_PATH = None

# Local
# CHECKS_PATH = 'style-transfer-checks.npz'

# Colab
#CHECKS_PATH = '/content/drive/My Drive/{}/{}'.format(FOLDERNAME, 'style-transfer-checks.npz')

assert CHECKS_PATH is not None, "[!] Choose path to style-transfer-checks.npz"

STYLES_FOLDER = CHECKS_PATH.replace('style-transfer-checks.npz', 'styles')

answers = dict(np.load(CHECKS_PATH))


Pytorch有两种不同的浮点数类型的张量：一种是放在CPU里面运算的 (`torch.FloatTensor`)，另一种是放在GPU上用CUDA运算的 (`torch.cuda.FloatTensor`)。
在以后我们将更多地使用此类型变量，因此我们需要将张量类型设置为其中之一。

In [ ]:
dtype = torch.FloatTensor
# Uncomment out the following line if you're on a machine with a GPU set up for PyTorch!
#dtype = torch.cuda.FloatTensor 

In [ ]:
# Load the pre-trained SqueezeNet model.
cnn = torchvision.models.squeezenet1_1(pretrained=True).features
cnn.type(dtype)

# We don't want to train the model any further, so we don't want PyTorch to waste computation 
# computing gradients on parameters we're never going to update.
for param in cnn.parameters():
    param.requires_grad = False

# Part 1: Computing Loss

现在，我们将计算损失函数的三个组成部分。 损失函数是三个项的加权之和：content loss + style loss + total variation loss。 你将在下面填写用于计算这些加权项的函数。

## Part 1A: Content loss

通过将两种图像都纳入我们的损失函数中，我们可以得到反映一幅图像内容和另一幅图​​像风格的生成图像。 我们要惩罚生成图像与内容图像在内容上的偏差，生成图像与风格图像在风格上的偏差。 然后，我们可以使用此混合损失函数来对生成图像的**像素值**执行梯度下降操作，**而不是对模型的参数**进行执行梯度下降。（也就是说在这里我们冻结了预训练模型的参数，只更新生成图片的像素值）

我们先写下 content loss 的计算公式。Content loss 衡量了生成图片与源图在特征图部分的差异大小。我们只关注网络中某一层的内容 (例如, layer $\ell$)，该层的特征图为 $A^\ell \in \mathbb{R}^{1 \times C_\ell \times H_\ell \times W_\ell}$。$C_\ell$ 为layer $\ell$ 中通道/filters的个数，$H_\ell$ 与 $W_\ell$ 为特征图的高度与宽度。接下来我们将特征图的每个通道各reshape成一维的向量。设 $F^\ell \in \mathbb{R}^{C_\ell \times M_\ell}$ 为生成图片的特征图且 $P^\ell \in \mathbb{R}^{C_\ell \times M_\ell}$ 为content源图的特征图，其中 $M_\ell=H_\ell\times W_\ell$ 为特征图中元素的个数。 $F^\ell$ or $P^\ell$ 中的每行代表向量化后的每个通道的特征图，这个特征图又是用网络对原图进行推理/卷积得到的。最后，$w_c$ 就是该 content loss 在总损失里的权重项了。

损失计算公式为:

$L_c = w_c \times \sum_{i,j} (F_{ij}^{\ell} - P_{ij}^{\ell})^2$

完成 `daseCV/style_transfer_pytorch.py` 中的 `content_loss`

测试你的 content loss 。 错误会小于0.001。

In [ ]:
from daseCV.style_transfer_pytorch import content_loss, extract_features, features_from_img

def content_loss_test(correct):
    content_image = '%s/tubingen.jpg' % (STYLES_FOLDER)
    image_size =  192
    content_layer = 3
    content_weight = 6e-2
    
    c_feats, content_img_var = features_from_img(content_image, image_size, cnn)
    
    bad_img = torch.zeros(*content_img_var.data.size()).type(dtype)
    feats = extract_features(bad_img, cnn)
    
    student_output = content_loss(content_weight, c_feats[content_layer], feats[content_layer]).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Maximum error is {:.3f}'.format(error))

content_loss_test(answers['cl_out'])

## Part 1B: Style loss

现在我们来处理 style loss。对于给定的 layer $\ell$，其 style loss 被定义如下：

首先，计算Gram矩阵 $G$，该矩阵代表特征图的每个通道中的值之间的相关性（即负责该通道的filter的“响应”强弱），其中 $F$ 的定义同上。Gram矩阵可以近似理解为协方差矩阵————它告诉我们每个通道中的值（即该filter的激活强度）与其他每个通道的值的相关性。如果我们有 $C$ 个通道，那矩阵 $G$ 的 shape 为 $(C, C)$ 以此获取这些相关性。

我们希望生成图像的激活统计信息与风格图像的激活统计信息匹配，并且匹配（近似）协方差是实现此目的的一种方法。 你可以通过多种方式来执行此操作，但是Gram矩阵很不错，因为它易于计算，并且在实践中可以显示出良好的效果。

给定一个 shape 为 $(C_\ell, H_\ell, W_\ell)$ 的特征图 $F^\ell$，我们可以将其摊平为 $M_\ell = H_\ell \times W_\ell$ 的一维向量：$F^\ell$ 的新的 shape 为 $(C_\ell, M_\ell)$ 。这样，Gram矩阵的 shape 就是 $(C_\ell, C_\ell)$，其中每个元素的计算公式为：

$$G_{ij}^\ell  = \sum_k F^{\ell}_{ik} F^{\ell}_{jk}$$

假设 $G^\ell$ 是生成图片的特征图的Gram矩阵， $A^\ell$ 是源图的特征图的Gram矩阵， 而且 $w_\ell$ 为标量权重项，那么 layer $\ell$ 的 style loss 就是这两个Gram矩阵的加权欧几里得距离：

$$L_s^\ell = w_\ell \sum_{i, j} \left(G^\ell_{ij} - A^\ell_{ij}\right)^2$$

通常在实际应用中我们常常计算一个 layers $\mathcal{L}$ 集合的 style loss；那么总的 style loss 就是每层的 style losses 的总和：

$$L_s = \sum_{\ell \in \mathcal{L}} L_s^\ell$$

首先在 `daseCV\style_transfer_pytorch.py` 中实现Gram矩阵计算函数 `gram_matrix` ：

测试你的Gram矩阵代码。 你应该看到小于0.001的错误。

In [ ]:
from daseCV.style_transfer_pytorch import gram_matrix
def gram_matrix_test(correct):
    style_image = '%s/starry_night.jpg' % (STYLES_FOLDER)
    style_size = 192
    feats, _ = features_from_img(style_image, style_size, cnn)
    student_output = gram_matrix(feats[5].clone()).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Maximum error is {:.3f}'.format(error))
    
gram_matrix_test(answers['gm_out'])

下一步，把他们放到一起并在 `daseCV/style_transfer_pytorch.py` 中实现 `style_loss` 函数

测试你的 style loss 。错误应该小于0.001.

In [ ]:
from daseCV.style_transfer_pytorch import style_loss
def style_loss_test(correct):
    content_image = '%s/tubingen.jpg' % (STYLES_FOLDER)
    style_image = '%s/starry_night.jpg' % (STYLES_FOLDER)
    image_size =  192
    style_size = 192
    style_layers = [1, 4, 6, 7]
    style_weights = [300000, 1000, 15, 3]
    
    c_feats, _ = features_from_img(content_image, image_size, cnn)    
    feats, _ = features_from_img(style_image, style_size, cnn)
    style_targets = []
    for idx in style_layers:
        style_targets.append(gram_matrix(feats[idx].clone()))
    
    student_output = style_loss(c_feats, style_layers, style_targets, style_weights).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Error is {:.3f}'.format(error))

    
style_loss_test(answers['sl_out'])

## Part 1C: Total-variation regularization

事实证明，鼓励图像变得平滑会对生成美观的图片有所帮助。 我们可以通过在损失中添加另一个项来惩罚像素值的摆动或“总变化”来做到这一点。

你可以将“总变化”定义为彼此相邻（水平或垂直）的所有像素对的像素值的欧几里得距离总和。在这里，我们对3个输入通道（RGB）中的每个输入通道的总变化求和，并用总变化权重 $w_t$ 对 total summed loss 进行加权：

$L_{tv} = w_t \times \left(\sum_{c=1}^3\sum_{i=1}^{H-1}\sum_{j=1}^{W} (x_{i+1,j,c} - x_{i,j,c})^2 + \sum_{c=1}^3\sum_{i=1}^{H}\sum_{j=1}^{W - 1} (x_{i,j+1,c} - x_{i,j,c})^2\right)$

在 `cs231/style_transfer_pytorch.py` 中的 `tv_loss` 函数中完成 TV（Total Variation） loss 项。为了得到所有作业分，你的代码不能出现循环。

测试你的 TV loss。错误要小于 0.0001。

In [ ]:
from daseCV.style_transfer_pytorch import tv_loss
from inspect import getsourcelines
import re

def tv_loss_test(correct):
    content_image = '%s/tubingen.jpg' % (STYLES_FOLDER)
    image_size =  192
    tv_weight = 2e-2

    content_img = preprocess(PIL.Image.open(content_image), size=image_size).type(dtype)
    
    student_output = tv_loss(content_img, tv_weight).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Error is {:.4f}'.format(error))
    lines, _ = getsourcelines(tv_loss)
    used_loop = any(bool(re.search(r"for \S* in", line)) for line in lines)
    if used_loop:
        print("WARNING!!!! - Your implementation of tv_loss contains a loop! To receive full credit, your implementation should not have any loops")
    
tv_loss_test(answers['tv_out'])

# Part 2: Style Transfer

现在我们可以把它们都串起来了（请仔细查看以下步骤确保理解整个过程，但是不要修改里面的内容）：

In [ ]:
def style_transfer(content_image, style_image, image_size, style_size, content_layer, content_weight,
                   style_layers, style_weights, tv_weight, init_random = False):
    """
    Run style transfer!
    
    Inputs:
    - content_image: filename of content image
    - style_image: filename of style image
    - image_size: size of smallest image dimension (used for content loss and generated image)
    - style_size: size of smallest style image dimension
    - content_layer: layer to use for content loss
    - content_weight: weighting on content loss
    - style_layers: list of layers to use for style loss
    - style_weights: list of weights to use for each layer in style_layers
    - tv_weight: weight of total variation regularization term
    - init_random: initialize the starting image to uniform random noise
    """
    
    # Extract features for the content image
    content_img = preprocess(PIL.Image.open(content_image), size=image_size).type(dtype)
    feats = extract_features(content_img, cnn)
    content_target = feats[content_layer].clone()

    # Extract features for the style image
    style_img = preprocess(PIL.Image.open(style_image), size=style_size).type(dtype)
    feats = extract_features(style_img, cnn)
    style_targets = []
    for idx in style_layers:
        style_targets.append(gram_matrix(feats[idx].clone()))

    # Initialize output image to content image or nois
    if init_random:
        img = torch.Tensor(content_img.size()).uniform_(0, 1).type(dtype)
    else:
        img = content_img.clone().type(dtype)

    # We do want the gradient computed on our image!
    img.requires_grad_()
    
    # Set up optimization hyperparameters
    initial_lr = 3.0
    decayed_lr = 0.1
    decay_lr_at = 180

    # Note that we are optimizing the pixel values of the image by passing
    # in the img Torch tensor, whose requires_grad flag is set to True
    optimizer = torch.optim.Adam([img], lr=initial_lr)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[0].set_title('Content Source Img.')
    axarr[1].set_title('Style Source Img.')
    axarr[0].imshow(deprocess(content_img.cpu()))
    axarr[1].imshow(deprocess(style_img.cpu()))
    plt.show()
    plt.figure()
    
    for t in range(200):
        if t < 190:
            img.data.clamp_(-1.5, 1.5)
        optimizer.zero_grad()

        feats = extract_features(img, cnn)
        
        # Compute loss
        c_loss = content_loss(content_weight, feats[content_layer], content_target)
        s_loss = style_loss(feats, style_layers, style_targets, style_weights)
        t_loss = tv_loss(img, tv_weight) 
        loss = c_loss + s_loss + t_loss
        
        loss.backward()

        # Perform gradient descents on our image values
        if t == decay_lr_at:
            optimizer = torch.optim.Adam([img], lr=decayed_lr)
        optimizer.step()

        if t % 100 == 0:
            print('Iteration {}'.format(t))
            plt.axis('off')
            plt.imshow(deprocess(img.data.cpu()))
            plt.show()
    print('Iteration {}'.format(t))
    plt.axis('off')
    plt.imshow(deprocess(img.data.cpu()))
    plt.show()

## Generate some pretty pictures!

在下面的三个不同的参数集上尝试 `style_transfer` 。 确保运行所有三个单元。 你可以随意添加自己的参数，但请确保在提交的notebook中的第三个参数集（星月夜）中包含 style transfer 的结果。

* The `content_image` is the filename of content image.
* The `style_image` is the filename of style image.
* The `image_size` is the size of smallest image dimension of the content image (used for content loss and generated image).
* The `style_size` is the size of smallest style image dimension.
* The `content_layer` specifies which layer to use for content loss.
* The `content_weight` gives weighting on content loss in the overall loss function. Increasing the value of this parameter will make the final image look more realistic (closer to the original content).
* `style_layers` specifies a list of which layers to use for style loss. 
* `style_weights` specifies a list of weights to use for each layer in style_layers (each of which will contribute a term to the overall style loss). We generally use higher weights for the earlier style layers because they describe more local/smaller scale features, which are more important to texture than features over larger receptive fields. In general, increasing these weights will make the resulting image look less like the original content and more distorted towards the appearance of the style image.
* `tv_weight` specifies the weighting of total variation regularization in the overall loss function. Increasing this value makes the resulting image look smoother and less jagged, at the cost of lower fidelity to style and content. 

在接下来的三个代码单元（不要更改超参数）的下方，你可以随时复制和粘贴参数并查看生成的图像的变化。

In [ ]:
# Composition VII + Tubingen
params1 = {
    'content_image' : '%s/tubingen.jpg' % (STYLES_FOLDER),
    'style_image' : '%s/composition_vii.jpg' % (STYLES_FOLDER),
    'image_size' : 192,
    'style_size' : 512,
    'content_layer' : 3,
    'content_weight' : 5e-2, 
    'style_layers' : (1, 4, 6, 7),
    'style_weights' : (20000, 500, 12, 1),
    'tv_weight' : 5e-2
}

style_transfer(**params1)

In [ ]:
# Scream + Tubingen
params2 = {
    'content_image':'%s/tubingen.jpg' % (STYLES_FOLDER),
    'style_image':'%s/the_scream.jpg' % (STYLES_FOLDER),
    'image_size':192,
    'style_size':224,
    'content_layer':3,
    'content_weight':3e-2,
    'style_layers':[1, 4, 6, 7],
    'style_weights':[200000, 800, 12, 1],
    'tv_weight':2e-2
}

style_transfer(**params2)

In [ ]:
# Starry Night + Tubingen
params3 = {
    'content_image' : '%s/tubingen.jpg' % (STYLES_FOLDER),
    'style_image' : '%s/starry_night.jpg' % (STYLES_FOLDER),
    'image_size' : 192,
    'style_size' : 192,
    'content_layer' : 3,
    'content_weight' : 6e-2,
    'style_layers' : [1, 4, 6, 7],
    'style_weights' : [300000, 1000, 15, 3],
    'tv_weight' : 2e-2
}

style_transfer(**params3)

# Part 3: Feature Inversion

你编写的代码还可以做另外一件很酷的事情。 为了理解卷积网络在识别图片的时候学会了哪些特征类型，最近的论文 "[Understanding Deep Image Representations by Inverting Them](https://arxiv.org/pdf/1412.0035.pdf)" 试图通过网络的特征来重建图像。我们可以使用来自预训练网络的图像梯度轻松实现此想法，这正是我们上面所做的（但是是两种不同的特征表示）。

现在，如果将 style weights 全部设置为0并将初始图像初始化为随机噪声（做风格迁移的时候初始化的生成图片可以是内容图片也可以是噪声图片），你将从内容源图像的特征表示中重建图像。 完全从噪声图片开始，但最终你会得到看起来像原始内容图像的图像。

（类似地，如果将 content weight 设置为0并将初始图像初始化为随机噪声，则可以从头开始进行“纹理合成”，但是在这里我们不要求你这样做。）

运行以下单元以尝试实现 feature inversion。

[1] Aravindh Mahendran, Andrea Vedaldi, "Understanding Deep Image Representations by Inverting Them", CVPR 2015


In [ ]:
# Feature Inversion -- Starry Night + Tubingen
params_inv = {
    'content_image' : '%s/tubingen.jpg' % (STYLES_FOLDER),
    'style_image' : '%s/starry_night.jpg' % (STYLES_FOLDER),
    'image_size' : 192,
    'style_size' : 192,
    'content_layer' : 3,
    'content_weight' : 6e-2,
    'style_layers' : [1, 4, 6, 7],
    'style_weights' : [0, 0, 0, 0], # we discard any contributions from style to the loss
    'tv_weight' : 2e-2,
    'init_random': True # we want to initialize our image to be random
}

style_transfer(**params_inv)